## FastAPI for Deployment

FastAPI is a [recent backend framework](https://fastapi.tiangolo.com/features/) that can help you deploy your model.

In `Deployment.py` we have:

In [ ]:
from fastapi import FastAPI
from typing import List
from pydantic import BaseModel
from pydantic import Field
import joblib
import numpy as np

app = FastAPI()

# Load the pre-trained model
loaded_pipeline = joblib.load('pipeline.joblib')

@app.get("/")                    
def read_items(): 
   return "Welcome to the CMP-3.5 Model"

# Input Schema
class Flower(BaseModel):
    feats: List[float] = Field(..., example="[1.0, 2.0, 3.0, 4.0]")

# Output Schema
class Prediction(BaseModel):
    pred : str = Field(..., title="Flower Prediction", example="Setosa")


@app.post("/predict_flower", response_model=Prediction)         # POST to /predict_flower
def predict_flower(featuresObj: Flower):
    try:
        # Convert input features to numpy array
        x = np.array(featuresObj.feats)
        x = x[np.newaxis, :]  # Reshape for prediction
        pred = loaded_pipeline.predict(x)
        return {"pred": pred[0]}
    except Exception as e:
        return {"error": str(e)}

Run `uvicorn Deployment:app --reload` and let's test it below.

In [1]:
from fastapi.testclient import TestClient
from Deployment import app

client = TestClient(app)

def test_predict_flower():
    # Test case 1: Valid input features
    response = client.post("/predict_flower", json={"feats": [5.1, 3.5, 1.4, 0.2]})
    assert response.status_code == 200
    assert "pred" in response.json()
    print(response.json())
    
if __name__ == "__main__":
    test_predict_flower()

{'pred': '0'}


It's straightforward steps to then deploy `Deployment.py` on Azure, AWS, GCP, PythonAnywhere, etc.